### The Memory Palace

Idea for the website: Make the user click anywhere, and the words of the poem appear randomly on the page. Maybe make them move to their place from a random position, or simply make them light up like a network of meanings (guirlande). Might make the words network by taking minimum cos similarity of word embeddings, then propagating in waves from the root, or normalize embeddings then take nearest neighbors in waves. (sort in increasing order of distance from the root). When the user hovers on a word, this becomes the new root and scrolling down reveals the words from the root. 

In [17]:
#!python3 -m spacy download en_core_web_md

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 2.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [18]:
import spacy
# Load the spacy model that you have installed
nlp = spacy.load('en_core_web_md')
# process a sentence using the model
doc = nlp("This is some text that I am processing with Spacy")

In [117]:
import numpy as np
import nltk
import json

nlp = spacy.load('en_core_web_md')

def create_embeddings(nlp, text):
    doc = nlp(text)
    embeddings = {}
    for token in doc:
        embeddings[token.text.lower()] = token.vector
    for key in embeddings.keys():
        norm = np.linalg.norm(embeddings[key])
        if norm > 0: 
            embeddings[key] = embeddings[key] / norm
    return embeddings

with open('memorypalace.txt', 'r') as fp:
    poem = fp.read()
print(poem)
embeddings = create_embeddings(nlp, poem)

with open('embeddings.json', 'w') as fp:
    embeds = {k: v.tolist() for k, v in embeddings.items()}
    json.dump(embeds, fp)


The Memory Palace

I. Along the corridor

An invitation to madness inscribed on a luminous mirror;
I watch my thousand-fold image on both sides stretching
as the ringing voice of a hunch-backed woman
reaches me with steady rhythm, pulsating, prowling

Her ancient face buried among deep furs,
knots of fingers dissimulate a golden key
and by a mute word orders me to follow
Along a corridor adorned with familiar sights;

A printing press oozing with black goo
An artificial greenhouse, its earth bearing
the unspeakable scars of an unruly beast
Crying mandragoras, pulled by their tuberculosis hair
A guitar lesson for mechanical girls, undeserving
An ornate clock rewound by blindfolded Fate

One grotesque idea standing amidst
a lavish interior of crimson cushions
‘See that red wine stain on her shirt?’
Bitemarks on an unripe fruit pleasing unknown people

Screeches, vain outpourings
of abandoned silence and laborious love
hiding behind gauzes of silky smoke
— ‘Back to the gate please’

Led b

In [115]:
def create_html_from_text(input_file, output_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    html_lines = []
    stanza_open = False

    for line in lines:
        line = line.strip()
        if line:
            if not stanza_open:
                html_lines.append(f'\t\t<div class="stanza">')
                stanza_open = True

            html_line = f'\t\t\t<p class="line">\n'
            words = line.split(' ')
            for word in words:
                html_line += f'\t\t\t\t<span class="word">{word}</span>\n'
            html_line += '\t\t\t</p>'
            html_lines.append(html_line)
        else:
            if stanza_open:
                html_lines.append(f'\t\t</div>')
                stanza_open = False

    if stanza_open:
        html_lines.append('</div>')

    html_content = '\n'.join(html_lines)

    html_template = f'''<!DOCTYPE html>
    <html lang="en">
    <body>
        {html_content}
    </body>
    </html>'''

    with open(output_file, 'w') as file:
        file.write(html_template)

create_html_from_text('memorypalace.txt', 'pages/poems/palace.html')


In [10]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")
model = Word2Vec.load("word2vec.model")

In [13]:
model.wv['computer']

array([-0.00515774, -0.00667028, -0.0077791 ,  0.00831315, -0.00198292,
       -0.00685696, -0.0041556 ,  0.00514562, -0.00286997, -0.00375075,
        0.0016219 , -0.0027771 , -0.00158482,  0.0010748 , -0.00297881,
        0.00852176,  0.00391207, -0.00996176,  0.00626142, -0.00675622,
        0.00076966,  0.00440552, -0.00510486, -0.00211128,  0.00809783,
       -0.00424503, -0.00763848,  0.00926061, -0.00215612, -0.00472081,
        0.00857329,  0.00428459,  0.0043261 ,  0.00928722, -0.00845554,
        0.00525685,  0.00203994,  0.0041895 ,  0.00169839,  0.00446543,
        0.0044876 ,  0.0061063 , -0.00320303, -0.00457706, -0.00042664,
        0.00253447, -0.00326412,  0.00605948,  0.00415534,  0.00776685,
        0.00257002,  0.00811905, -0.00138761,  0.00808028,  0.0037181 ,
       -0.00804967, -0.00393476, -0.0024726 ,  0.00489447, -0.00087241,
       -0.00283173,  0.00783599,  0.00932561, -0.0016154 , -0.00516075,
       -0.00470313, -0.00484746, -0.00960562,  0.00137242, -0.00

In [14]:
poem = "On the final floor; massed under the moonlit dome\nchaos thundering over a circular courtroom, \nan autodafé of ecstatic Javanese mouths gaping at you\nthe overturned martyr, levitating over a central pit"
tokens = nltk.word_tokenize(poem)
embeddings = [model.wv[token.lower()] for token in tokens]


KeyError: "Key 'on' not present"